In [ ]:
import time
import warnings
import numpy as np
import pandas as pd

from surprise import NormalPredictor
from surprise import BaselineOnly
from surprise import KNNBasic
from surprise import KNNWithMeans
from surprise import KNNBaseline
from surprise import SVD
from surprise import SVDpp
from surprise import NMF
from surprise import SlopeOne
from surprise import CoClustering
from surprise import Dataset
from surprise import evaluate
from surprise import print_perf
from surprise import Reader

warnings.filterwarnings('ignore')
%matplotlib inline

In [ ]:
# Start of the algorithm
t_start = time.time()

# Read data

# Build the recommender system

In [ ]:
# get the total code used time information
t_end = time.time()
print("Program running time\n")
print('Start time:\t{0:15.2f}'.format(t_start))
print('Stop  time:\t{0:15.2f}'.format(t_end))
print('Total time:\t{0:15.2f} seconds'.format(t_end - t_start))